In [ ]:
import numpy as np
import sklearn.datasets
import matplotlib.pyplot as plt
import statsmodels.api as sm
from statsmodels.api import add_constant

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_curve, roc_auc_score, make_scorer
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

### Breast Cancer Dataset Explanation

The **Breast Cancer dataset** is a widely-used binary classification dataset that comes from the **sklearn.datasets** module. It contains **569 samples** of data on breast cancer patients, with each sample corresponding to a patient's tumor. Each sample is described by **30 numeric features**, which represent various characteristics of the cell nuclei present in a biopsy. These features include attributes such as the **mean radius**, **texture**, **perimeter**, **area**, and **smoothness** of the cell nuclei.

The target variable is binary, indicating whether the cancer is **benign (label 1)** or **malignant (label 0)**. The goal of the dataset is to predict whether a given tumor is benign or malignant based on the input features. It is a well-balanced dataset with **357 benign** and **212 malignant** instances.

In addition to the feature data and target labels, the **Breast Cancer dataset** object from `sklearn` contains the following useful components:

1. **`data`:** A 2D array of shape (569, 30) containing the feature values for each sample (30 features per sample).
2. **`target`:** A 1D array of shape (569,) with the binary target values (0 for malignant, 1 for benign).
3. **`target_names`:** The array `['malignant', 'benign']`, which provides a label for each target class.
4. **`feature_names`:** A list of 30 feature names describing the measurements of the cell nuclei (e.g., 'mean radius', 'mean texture').
5. **`DESCR`:** A detailed description of the dataset, including information about the features and the source of the dataset.
6. **`filename`:** The path to the location of the dataset file, if applicable.

These components provide all the necessary details to understand and use the dataset effectively for classification tasks.

# Analysis Plan

### Objective
The objective is to estimate the logistic regression coefficients $\widehat\beta$, evaluate the model, and interpret the associated odds ratios.

### Steps

1. **Load the data and metadata**:
   - Import the dataset and corresponding metadata, ensuring that data types and variable descriptions are clear.

2. **Identify features and build the feature matrix (X-matrix)**:
   - Select relevant predictor variables (features) from the dataset.
   - Construct the feature matrix $X$ that will be used as input for the logistic regression model.

3. **Identify the binary outcome (response variable)**:
   - Define the binary response variable, $y$, which will serve as the target for the logistic regression function.

4. **Variable selection**:
   - Implement a method for feature selection (e.g., stepwise selection, LASSO, or other regularisation techniques).
   - Differentiate between significant and insignificant features based on their contribution to the model (e.g., using p-values, AIC/BIC criteria, or cross-validation).

5. **Estimate $\widehat\beta$ and evaluate model performance**:
   - Fit the logistic regression model and estimate the coefficients $\widehat\beta$.
   - Evaluate the performance of the model using metrics such as accuracy, precision, recall, F1-score, and the area under the ROC curve (AUC).

6. **Derive odds ratios and interpret results**:
   - Calculate the odds ratios from $\widehat\beta$ using the transformation $\text{Odds Ratio} = e^{\widehat{\beta}}$.
   - Interpret the odds ratios to assess the impact of each predictor on the likelihood of the outcome.

## 1. **Load the data and metadata**

To load the dataset, we can use `sklearn.datasets.load_breast_cancer()`. This function loads the breast cancer dataset, which includes both the features (predictors) and the binary outcome (response). It also contains additional metadata that can help describe the dataset, such as feature names, target names, and a full dataset description accessible via the `DESCR` attribute. This information wa obtained after googling this function and reading the documentation at:
https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html

In [ ]:
cancer = sklearn.datasets.load_breast_cancer()
print(cancer["DESCR"])

## 2. **Identify features and build the feature matrix (X-matrix)**

The `cancer.data` contains a numpy array with 569 observations (samples) and 30 features (predictors). These features represent various measurements of tumour cells. The feature matrix $X$ will consist of these 30 features for all observations.

## 3. **Identify the Binary Outcome (Response Variable)**

The `cancer.target` contains 569 entries, where:
- `0` corresponds to a benign tumour,
- `1` corresponds to a malignant tumour.

This binary response will be used as the target variable $y$ for logistic regression.

In [ ]:
x = cancer.data
print(x.shape)
y = cancer.target
print(y.shape)
print(y[:30])

## 4. **Variable Selection Using LASSO with Cross-Validation**

### Key Steps:
   - Use `sklearn.linear_model.LogisticRegressionCV` to perform LASSO with cross-validation on the feature matrix $X$.
   - The cross-validation will select the optimal penalty coefficient (denoted by `C` in the pacage that corresponds to $\lambda$) that maximises model performance, using the area under the ROC curve (AUC) as the evaluation metric.
   - After LASSO, irrelevant features will have a coefficient of zero, and relevant features will have non-zero coefficients.

In [ ]:
seed = 1 
cv_lambdas = np.logspace(-4, 0, 100)

cv_folds = 5
lasso_cv = LogisticRegressionCV(
    Cs=cv_lambdas,  # Number of inverse regularisation strength values to test (same as `lambda` in the lecture notes)
    cv=cv_folds,  # 5-fold cross-validation
    penalty='l1',  # LASSO (L1 regularisation)
    solver='saga',  # Solver that supports L1 regularisation
    scoring='roc_auc',  # Optimise for AUC
    max_iter=10000,  # Allow for sufficient iterations for convergence
    random_state=seed,
)

lasso_cv.fit(x, y)

mean_auc_scores = np.mean(lasso_cv.scores_[1], axis=0)
se_auc_scores = np.std(lasso_cv.scores_[1], axis=0)/np.sqrt(cv_folds-1)

plt.figure(figsize=(8, 6))
plt.errorbar(
    cv_lambdas, 
    mean_auc_scores, 
    yerr=se_auc_scores, 
    marker='o', 
    color='blue', 
    label='Mean AUC',
)
plt.xscale('log')  # Use log scale for C values (since C spans several magnitudes)
plt.xlabel(r'$\lambda$')
plt.ylabel('Mean AUC')
plt.title(r'AUC vs $\lambda$ in LASSO Logistic Regression')
plt.grid(True)

max_auc_index = np.argmax(mean_auc_scores)
max_auc = mean_auc_scores[max_auc_index]
max_lambda = cv_lambdas[max_auc_index]

plt.axvline(x=max_lambda, color='r', linestyle='--', label = fr"$\lambda_{{max}}={{{max_lambda:0.2}}}$")
plt.legend(loc="best")

In [ ]:
beta_hat_lambda_max = lasso_cv.coef_
beta_hat_lambda_max

5. **Estimate $\widehat\beta$ and evaluate model performance**:
   - Since LASSO tends to underestimate the coefficients due to the penalisation, you'll use the selected variables (non-zero coefficients) to refit a logistic regression model.
   - This will provide a better estimate of $\widehat\beta$ for the significant predictors.
   - For evaluation we will use the average AUC from a nested cross-validation evaluation.

In [ ]:
mask = np.abs(lasso_cv.coef_[0]) != 0

x_support = x[:, mask]
x_support = add_constant(x_support)

glm_train = sm.GLM(y , x_support, family=sm.families.Binomial())
results = glm_train.fit()

probs = results.predict(exog=x_support)

beta_hat = results.params

print(f"intercept: {beta_hat[0]:.3}")
for coef, feature_name in zip(beta_hat[1:], cancer.feature_names[mask]):
    print(f"{feature_name}: {coef:.3}")


## Nested cross-validation evaluation.

In [ ]:
outer_cv = KFold(n_splits=5, shuffle=True, random_state=0)
inner_cv = KFold(n_splits=10, shuffle=True, random_state=0)

lasso_pipeline = Pipeline([
    ('logreg', LogisticRegressionCV(cv=inner_cv, penalty='l1', solver='saga', max_iter=10000, scoring='roc_auc'))  # Lasso-style regularization
])

nested_scores = cross_val_score(lasso_pipeline, x, y, cv=outer_cv, scoring='roc_auc')

print(f"Nested CV AUC Scores: {nested_scores}")
print(f"Mean AUC: {np.mean(nested_scores):.4f}")

The mean AUC is very close to 1 therefore, these features are very good in predicting the outcome of this data.

6. **Derive odds ratios and interpret results**:
   - Calculate the odds ratios from $\widehat\beta$ using the transformation $\text{Odds Ratio} = e^{\widehat{\beta}}$.
   - Interpret the odds ratios to assess the impact of each predictor on the likelihood of the outcome.

In [ ]:
np.exp(beta_hat[1:])

### Interpreting the Odds Ratios:

1. **Odds Ratio for Mean Area (1.02076911)**:
   - The odds ratio of **$\sim1.02$** for **mean area** shows that for each unit increase in the mean area of a tumour, the odds of having a malignant tumour increase by **2%**.
   - This means that larger tumours, are associated with a slightly higher probability of being malignant.

2. **Odds Ratio for Worst Area (0.97356275)**:
   - The odds ratio of approximately **0.97** for **worst area** indicates that for each unit increase in the worst area of a tumour, the odds of having a malignant tumour **decrease** by **2.64%**.
   - Contrary to the mean area, an increase in the worst area of a tumour (the area of the largest tumour detected) is associated with a **slight decrease** in the likelihood of malignancy.

### General Rule for Interpreting Odds Ratios:
- **Odds ratio > 1**: A positive association, meaning that as the predictor increases, the odds of the outcome increase.
- **Odds ratio < 1**: A negative association, meaning that as the predictor increases, the odds of the outcome decrease.
- **Odds ratio = 1**: No effect, meaning the predictor does not influence the odds of the outcome.